# Cost Sensitive Analysis Structure

We can predetermine each transactions operational cost based on our models fraud prediction for the transaction:
* Legitimate Transactions (false negative, FN) - no cost, transactions passes through the model
* Misclassified as Fraud (false positive, FP) - ReviewCost
* Undetected fraud (false positive, FP) - amount of transaction (successful fraud attempt) & chargeback fee
* Detected fraud (true positive, TP) - ReviewCost

These potential costs can be written to a cost matrix that has the same number of rows as our dataset. The cost matrix is then applied to the predicted results using the following logic. The associated Boolean Truth tables demonstate how the logic isolates the transactions for each of the four possible matrix outcomes (FN, FP, FP, TP):

<img src="../images/cost_matrix_formulas.jpg" alt="Drawing" style="width: 600px;"/>

The function cs_confusion_matrix uses these formulas to construct a cost-sensitive confusion matrix which oulines the total operation costs, including misclassification costs, associated with our fraud prediction model.

In [ ]:
def cs_confusion_matrix(y_test, y_pred, cost_matrix):
    '''
    Returns a cost sensitive confusion matrix using the cost matrix
    '''
    cost_TN = np.sum((1 - y_test) * (1 - y_pred) * cost_matrix[:, 0])
    cost_FP = np.sum((1 - y_test) * y_pred * cost_matrix[:, 1])
    cost_FN = np.sum(y_test * (1 - y_pred) * cost_matrix[:, 2])
    cost_TP = np.sum(y_test * y_pred * cost_matrix[:, 3])
    return np.array([[cost_TN, cost_FP],[cost_FN, cost_TP]])

### Build Cost Matrix based on the Fraud Management Operational Strategy
Once we identify the transactions suspected to be fraudulent, they will be placed in a Fraud Review Queue to be manually inspected by fraud analysts to determine their legitimacy and those that are found to be fraud will be cancelled. There are multiple approaches to how this fraud review process can be managed. One approach would be to review all suspected fraud transactions while another approach would be to automatically cancel some of the suspected fraud without manaully reviewing the transactins. These different operatinal strategies have different impacts on our fraud losses, operational expenses and customer satisfaction. By building different cost matrix, we can measure what the impacts would be from employing different operational strategies. 

We will start by optimizing a model based on a manual review of all suspected fraud transactions and then, once this model is build, we will test the impact of an auto-decline strategy where we automatically decline suspected fraud transactions with dollar amounts that exceed our average cost of reviewing a transaction.

**Full Review model costs**:
- False negative - Undetected fraud transactions = Transaction Amount plus a ChargebackFee assessed by our credit card processor
- True positive - Correctly classified fraud transactions = ReviewCost
- False positive - Misclassified Legitimate transactions = ReviewCost

<img src="../images/cost_matrix1.jpg" alt="Drawing" style="width: 350px;"/>

In [ ]:
def FullReview(c):
    '''
    Returns a cost_matrix that contains, for every transaction, the four possible costs associated with it depending
    on the outcome of its classification (TN, FP, FN, TP). The function must be an array c which contains
    the amount of each transaction. The model assumes we will manually review all suspected fraud transactions
    and cancel those in which fraud is confirmed.     
    ''' 
    n_samples = c.shape[0]
    cost_matrix = np.zeros((n_samples, 4))
    # True Negative: correctly predicted as legitimat
    cost_matrix[:, 0] = 0.0
    
    # False Positive: incorrectly predicted as fraud.
    cost_matrix[:, 1] = ReviewCost                                                               
    
    # False Negative: undetected fraud.
    cost_matrix[:, 2] = c + ChargebackFee 

    # False Negative: correctly prediced as fraud.
    cost_matrix[:, 3] = ReviewCost

    return cost_matrix 

**Partial Review Model**:
Suspected fraud transactions are only reviewed if their amount exceeds our ReviewCost. It this model we have a risk of cancelling some legitimate transactions. When this occurs we will loose the profit from these transactions which we will assume is 50 percent of their total amount. Therefore the costs associated with an auto-cancel model are:

- False negative - Undetected fraud transactions = Transaction Amount plus a ChargebackFee assessed by our credit card processor
- True positive - Correctly classified fraud transactions = ReviewCost when transaction amount > ReviewCost
- False positive - Misclassified Legitimate transactions = ReviewCost when transaction amount > ReviewCost, otherwise it is the profit from this lost sale which is 50 percent of the transaction amount.
<img src="../images/cost_matrix2.jpg" alt="Drawing" style="width: 350px;"/>

In [ ]:
def PartialReview(c):
    '''
    Returns a cost_matrix that contains, for every transaction, the four possible costs associated with it depending
    on the outcome of its classification (TN, FP, FN, TP). The function must be an array c which contains
    the amount of each transaction. The model assumes we will only manually review suspected fraud transactions
    with a value less than the ReviewCost. 
    ''' 
    n_samples = c.shape[0]
    cost_matrix = np.zeros((n_samples, 4))
    
    # True Negative: correctly predicted as legitimate
    cost_matrix[:, 0] = 0.0 
    
    # False Positive: incorrectly predicted as fraud. Reviewed only if transaction amount exceeds the review cost.
    # Cost: ReviewCost if reviewed, otherwise half of the transaction amount (estimated profit lost from cancellng trans)
    cost_matrix[:, 1] = np.where(c>=ReviewCost, ReviewCost, c*0.5)                                                               
    
    # False Negative: undetected fraud. Cost: Amount of transaction plus a chargeback fee
    cost_matrix[:, 2] = c + ChargebackFee 
    
    # False Negative: correctly prediced as fraud. Reviewed only if transaction amount exceeds the review cost.
    # Cost: ReviewCost if transaction is reviewed.
    cost_matrix[:, 3] = np.where(c>=ReviewCost, ReviewCost, 0)

    return cost_matrix 

**Auto-Cancel model**:

In [ ]:
def NoReview(c):
    '''
    Returns a cost_matrix that contains, for every transaction, the four possible costs associated with it depending
    on the outcome of its classification (TN, FP, FN, TP). The function must be an array c which contains
    the amount of each transaction. The model assumes all transaction suspected of fraud will be cancelled without
    a manual review.
    ''' 
    n_samples = c.shape[0]
    cost_matrix = np.zeros((n_samples, 4))
    
    # True Negative: correctly predicted as legitimate
    cost_matrix[:, 0] = 0.0 
    
    # False Positive: incorrectly predicted as fraud. Cost: half of the transaction amount (estimated profit lost
    # from cancellng trans)
    cost_matrix[:, 1] = c*0.5                                                             
    
    # False Negative: undetected fraud. Cost: Amount of transaction plus a chargeback fee
    cost_matrix[:, 2] = c + ChargebackFee 
    
    # False Negative: correctly prediced as fraud. 
    cost_matrix[:, 3] = 0.0

    return cost_matrix 